In [5]:
import pyhealth
import pickle
import os

## dataset lookups

The dataset synthesized from MIMIC-III contains several .pkl files for both the training
and test portion. We show the contents below.

In [6]:
"""
where

- `pids`: contains the patient ids
- `vids`: contains a list of visit ids for each patient
- `hfs`: contains the heart failure label (0: normal, 1: heart failure) for each patient
- `seqs`: contains a list of visit (in ICD9 codes) for each patient
- `types`: contains the map from ICD9 codes to ICD-9 labels
- `rtypes`: contains the map from ICD9 labels to ICD9 codes
"""

DATA_PATH = "./"
pids = pickle.load(open(os.path.join(DATA_PATH,'train/pids.pkl'), 'rb'))
vids = pickle.load(open(os.path.join(DATA_PATH,'train/vids.pkl'), 'rb'))
hfs = pickle.load(open(os.path.join(DATA_PATH,'train/hfs.pkl'), 'rb'))
seqs = pickle.load(open(os.path.join(DATA_PATH,'train/seqs.pkl'), 'rb'))
types = pickle.load(open(os.path.join(DATA_PATH,'train/types.pkl'), 'rb'))
rtypes = pickle.load(open(os.path.join(DATA_PATH,'train/rtypes.pkl'), 'rb'))

assert len(pids) == len(vids) == len(hfs) == len(seqs) == 1000
assert len(types) == 619

In [7]:
# take the 3rd patient as an example

print("Patient ID:", pids[3])
print("Heart Failure:", hfs[3])
print("# of visits:", len(vids[3]))
for visit in range(len(vids[3])):
    print(f"\t{visit}-th visit id:", vids[3][visit])
    print(f"\t{visit}-th visit diagnosis labels:", seqs[3][visit])
    print(f"\t{visit}-th visit diagnosis codes:", [rtypes[label] for label in seqs[3][visit]])

Patient ID: 47537
Heart Failure: 0
# of visits: 2
	0-th visit id: 0
	0-th visit diagnosis labels: [12, 103, 262, 285, 290, 292, 359, 416, 39, 225, 275, 294, 326, 267, 93]
	0-th visit diagnosis codes: ['DIAG_041', 'DIAG_276', 'DIAG_518', 'DIAG_560', 'DIAG_567', 'DIAG_569', 'DIAG_707', 'DIAG_785', 'DIAG_155', 'DIAG_456', 'DIAG_537', 'DIAG_571', 'DIAG_608', 'DIAG_529', 'DIAG_263']
	1-th visit id: 1
	1-th visit diagnosis labels: [12, 103, 240, 262, 290, 292, 319, 359, 510, 513, 577, 307, 8, 280, 18, 131]
	1-th visit diagnosis codes: ['DIAG_041', 'DIAG_276', 'DIAG_482', 'DIAG_518', 'DIAG_567', 'DIAG_569', 'DIAG_599', 'DIAG_707', 'DIAG_995', 'DIAG_998', 'DIAG_V09', 'DIAG_584', 'DIAG_031', 'DIAG_553', 'DIAG_070', 'DIAG_305']


In [8]:
print("number of heart failure patients:", sum(hfs))
print("ratio of heart failure patients: %.2f" % (sum(hfs) / len(hfs)))

number of heart failure patients: 548
ratio of heart failure patients: 0.55


## Step1 & 2: dataset wrapper and split

- Since we already have a clean dataset, we need to structure it into the PyHealth format. 
- As we have learned from Section 2.3.2 that the data samples in
PyHealth are stored in json structure, the following code will essentially organize the
patient ID, visit ID, label, and diagnosis information from this heart failure dataset
into one JSON structure, subsequently adding it to the samples list.
- We can then employ the pyhealth.dataset.SampleEHRDataset API to transform it into the PyHealth
supported structure.

In [9]:
"""
With this user processed data, we just need to wrap up the data to fit
the pyhealth format.

Task definition: we will use all the visits from the patients as the features,
    then, the labels are given b the hfs list.
"""
samples = []
for pid, vid, hf, seq in zip(pids, vids, hfs, seqs):
    samples.append(
        {
            'patient_id': pid,
            'visit_id': vid[-1],
            'label': hf,
            'diagnoses': [[rtypes[v] for v in visit] for visit in seq],
        }
    )
    

In [10]:
# load into the dataset
from pyhealth.datasets import SampleEHRDataset
train_dataset = SampleEHRDataset(samples, code_vocs=None)

/home/chaoqiy2/miniconda3/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [11]:
train_dataset.samples[0]

{'patient_id': 89571,
 'visit_id': 1,
 'label': 1,
 'diagnoses': [['DIAG_250',
   'DIAG_285',
   'DIAG_682',
   'DIAG_730',
   'DIAG_531',
   'DIAG_996',
   'DIAG_287',
   'DIAG_276',
   'DIAG_E878',
   'DIAG_V45',
   'DIAG_996'],
  ['DIAG_250',
   'DIAG_276',
   'DIAG_285',
   'DIAG_998',
   'DIAG_996',
   'DIAG_078',
   'DIAG_336',
   'DIAG_E878',
   'DIAG_401',
   'DIAG_205']]}

In [12]:
from pyhealth.datasets.splitter import split_by_patient
from pyhealth.datasets import split_by_patient, get_dataloader

# dataset split by patient id
train_ds, val_ds, test_ds = split_by_patient(train_dataset, [0.8, 0.2, 0])

# obtain train/val/test dataloader, they are <torch.data.DataLoader> object
train_loader = get_dataloader(train_ds, batch_size=64, shuffle=True)
val_loader = get_dataloader(val_ds, batch_size=64, shuffle=False)

## Step3: initialize the LSTM model

Right now, we obtained the training loader and the validation loader, we want to
initialize an LSTM model. We call the pyhealth.model.RN N API below. The API
uses the training dataset, set diagnoses key as the features, choose LSTM as the RNN
type, embedding dimension and the hidden dimension are both set at 64, and configure
other attributes on need. The initialization steps is similar to the DNN initialization in
the last section


In [13]:
from pyhealth.models import RNN

model = RNN(
    dataset=train_dataset,
    feature_keys=["diagnoses"],
    label_key="label",
    mode="binary",
    rnn_type="LSTM",
    num_layers=2,
    embedding_dim=64,
    hidden_dim=64,
)

## Step4: model training

Similar to previous pipelines, we use the pyhealth.trainer.Trainer to handle the
model training and evaluation. During the training, we record the precision-recall area
under curve (PRAUC), AUROC, and the F1 measure as the evaluation metrics. We
set the training epochs to 20 and monitor the AUROC curves to select the best model
based on the validation set. The best model will automatically be loaded in the Trainer
for the next evaluation step. The configuration of Trainer could refer to [this page](https://pyhealth.readthedocs.io/en/latest/api/trainer.html).

In [14]:
from pyhealth.trainer import Trainer

# use our Trainer to train the model

trainer = Trainer(
    model=model,
    metrics=["pr_auc", "roc_auc", "f1"]
)

trainer.train(
    train_dataloader=train_loader,
    val_dataloader=val_loader,
    epochs=20,
    monitor="roc_auc",
)

/home/chaoqiy2/data/MNIST/github/PyHealth/pyhealth/trainer.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


RNN(
  (embeddings): ModuleDict(
    (diagnoses): Embedding(601, 64, padding_idx=0)
  )
  (linear_layers): ModuleDict()
  (rnn): ModuleDict(
    (diagnoses): RNNLayer(
      (dropout_layer): Dropout(p=0.5, inplace=False)
      (rnn): LSTM(64, 64, num_layers=2, batch_first=True, dropout=0.5)
    )
  )
  (fc): Linear(in_features=64, out_features=1, bias=True)
)
Metrics: ['pr_auc', 'roc_auc', 'f1']
Device: cuda

Training:
Batch size: 64
Optimizer: <class 'torch.optim.adam.Adam'>
Optimizer params: {'lr': 0.001}
Weight decay: 0.0
Max grad norm: None
Val dataloader: <torch.utils.data.dataloader.DataLoader object at 0x7f6f60390370>
Monitor: roc_auc
Monitor criterion: max
Epochs: 20



Epoch 0 / 20:   0%|          | 0/13 [00:00<?, ?it/s]

--- Train epoch-0, step-13 ---
loss: 0.6874


Evaluation: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 164.56it/s]

...

--- Eval epoch-18, step-247 ---
pr_auc: 0.8038
roc_auc: 0.7843
f1: 0.7395
loss: 0.7313



Epoch 19 / 20:   0%|          | 0/13 [00:00<?, ?it/s]

--- Train epoch-19, step-260 ---
loss: 0.4679


Evaluation: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 147.10it/s]

--- Eval epoch-19, step-260 ---
pr_auc: 0.8077
roc_auc: 0.7866
f1: 0.7364
loss: 0.7543
Loaded best model


## Step5: model evaluation

So far, we have processed the training set into train and validation and obtain the
best trained LSTM model. Now, we will similarly process the test data into PyHealth
format, use the same pyhealth wrapper to get a test data loader and compute the final
evaluation metrics. The trainer.evaluate() function automatically use the best trained
LSTM model for evaluation

In [15]:
DATA_PATH = "./"
pids = pickle.load(open(os.path.join(DATA_PATH,'test/pids.pkl'), 'rb'))
vids = pickle.load(open(os.path.join(DATA_PATH,'test/vids.pkl'), 'rb'))
hfs = pickle.load(open(os.path.join(DATA_PATH,'test/hfs.pkl'), 'rb'))
seqs = pickle.load(open(os.path.join(DATA_PATH,'test/seqs.pkl'), 'rb'))
types = pickle.load(open(os.path.join(DATA_PATH,'test/types.pkl'), 'rb'))
rtypes = pickle.load(open(os.path.join(DATA_PATH,'test/rtypes.pkl'), 'rb'))

In [16]:
test_samples = []
for pid, vid, hf, seq in zip(pids, vids, hfs, seqs):
    test_samples.append(
        {
            'patient_id': pid,
            'visit_id': vid[-1],
            'label': hf,
            'diagnoses': [[rtypes[v] for v in visit] for visit in seq],
        }
    )
    
test_dataset = SampleEHRDataset(test_samples, code_vocs=None)
test_loader = get_dataloader(test_dataset, batch_size=64, shuffle=False)

In [17]:
result = trainer.evaluate(test_loader)
print (result)

Evaluation: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 149.98it/s]

{'pr_auc': 0.7620261511596843, 'roc_auc': 0.7703540432566468, 'f1': 0.7465753424657535, 'loss': 0.6352440267801285}
